# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6912649170bc8191a3da9b17c96aadda


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-AxnBDXJqCyrHPFXB1NRRKW


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-AxnBDXJqCyrHPFXB1NRRKW


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6010846063542131
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5987929345462765
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.591898195864025
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5622353955032359
🤤 AI is making cinematic food commercials in minutes. The sizzling patty, the dripping cheese... no 
 Relevant score: 0.551242779477055


## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of 2023, there have been several advancements in generative AI:

1. **Multimodal Models**: New models are capable of processing and generating text, images, audio, and video simultaneously, improving the integration of diverse data types.

2. **Larger and More Efficient Architectures**: Advances in model architecture have enabled the creation of even larger models while reducing computational requirements and improving efficiency.

3. **Ethical and Bias Mitigation**: Ongoing research has led to better techniques for minimizing bias and ensuring ethical use of generative AI systems.

4. **Real-Time Generation**: Generative AI systems can now produce content in real-time, opening opportunities for interactive applications and dynamic content generation.

5. **Domain-Specific Applications**: Specialized models have been developed for specific industries such as healthcare, finance, and entertainment, providing more tailored solutions.

6. **Improved Natural Language Understanding**: Language models have become better at understanding context, nuance, and intent, leading to more natural and coherent interactions.

7. **Enhanced Creativity Tools**: Tools leveraging generative AI are now more widely available for creative industries, enabling artists and designers to enhance their workflows with AI-generated content.

These developments reflect the rapid progress and expanding potential of generative AI across various fields.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are likely to enhance automation and efficiency in various applications.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risks and boost resilience.

3. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of cinematic videos and other content with minimal human intervention.

4. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing for the creation of custom large language models to enhance customer engagement and streamline processes.

5. **Market Growth**: The global generative AI market is expected to reach $1.18 billion this year, indicating rapid growth and adoption across various sectors.

These developments highlight the expanding role of generative AI in both business and creative fields, driving innovation and efficiency.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Here’s an in-depth overview of the **latest developments in generative AI** as of **November 10, 2025**. I’ve structured the summary into key themes—model releases, industrial applications, hardware advances, and societal implications—each richly referenced to ensure accuracy.

---

## 1. New Model Launches and Generative Capabilities

**OpenAI’s GPT‑5**  
- Released on **August 7, 2025**, GPT‑5 is OpenAI’s newest multimodal foundation model combining high-throughput and deep reasoning sub-models with an intelligent routing system. It enables agentic behavior such as autonomously using browsers and desktop tools. It is available via ChatGPT, Microsoft Copilot, and the OpenAI API ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

**Google’s Nano Banana (Gemini 2.5 Flash Image)**  
- Launched on **August 26, 2025**, this text-to-image editing model gained viral popularity for generating photorealistic 3D figurine-style images. It features subject consistency, multi-image fusion, and SynthID watermarking, and is integrated into the Gemini app, Google AI Studio, and Vertex AI ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

**Google DeepMind’s Veo 3.1**  
- Released its latest version on **October 15, 2025**. Veo 3—a text-to-video model introduced earlier in May 2025—generates synchronized video and audio (dialogue, sound effects, ambient noise). Veo 3.1 builds on these capabilities ([en.wikipedia.org](https://en.wikipedia.org/wiki/Veo_%28text-to-video_model%29?utm_source=openai)).

**Meta’s Generative Ads Model (GEM)**  
- Unveiled **today (November 10, 2025)**, GEM is a groundbreaking foundation model designed for ad recommendations. Trained at an LLM-like scale across thousands of GPUs, it significantly improves ad targeting performance on Facebook and Instagram by enhancing precision and scalability ([engineering.fb.com](https://engineering.fb.com/2025/11/10/ml-applications/metas-generative-ads-model-gem-the-central-brain-accelerating-ads-recommendation-ai-innovation/?utm_source=openai)).

---

## 2. Industrial & Market Applications

**TIME AI Agent**  
- Launched **today**, the TIME AI Agent is a unified generative platform developed with Scale AI. It enables personalized article summaries, translations, audio reports, and interactive news exploration. It emphasizes editorial integrity, data governance, and moderation—marking a shift in how journalism interfaces with AI ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai)).

**India Generative AI Market Outlook**  
- As of **2024**, India’s generative AI market stood at **USD 1.30 billion**, projected to grow at a **15.2% CAGR** toward **USD 5.40 billion** by **2033**. Key adoption areas include media, advertising, healthcare (drug discovery and medical imaging), manufacturing, logistics, and customer service. Ethical AI and responsible innovation are emphasized trends ([openpr.com](https://www.openpr.com/news/4261720/india-generative-ai-market-size-share-trends-growth-report?utm_source=openai)).

---

## 3. Hardware & Infrastructure Accelerators

**Qualcomm’s AI200 and AI250 Accelerators**  
- Announced around **late October 2025**, Qualcomm introduces two neural inference accelerators—AI200 (for 2026) and AI250 (for 2027)—to compete with AMD and Nvidia in data center AI deployments. They offer features like micro-tile inferencing, 64‑bit addressing, encrypted models, direct liquid cooling, and high-power racks, with full software support for frameworks like PyTorch and ONNX ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai)).

---

## 4. Broader Trends and Research Directions

**Real-Time Interactive 3D Worlds: Genie 3**  
- Released earlier in **2025**, Google DeepMind’s Genie 3 generates interactive 3D environments from text prompts with dynamic world-building, memory retention, and adjustable scenarios. It supports real-time interactivity at 720p and 24 fps with ethical and energy-conscious design ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/google-deepminds-genie-3-how-ai-instantly-builds-interactive-3d-worlds-from-a-single-text-prompt-ideal-for-gaming-and-education/articleshow/123140080.cms?utm_source=openai)).

**Emerging Trends in 2025**  
- Generative AI is becoming more multimodal, fusing text, images, audio, video, and even 3D content in unified frameworks ([codenomad.net](https://www.codenomad.net/blog/what-new-in-generative-ai/?utm_source=openai)).
- Small language models (SLMs) like Phi‑3, TinyLlama, and Mistral 7B are gaining traction for privacy-preserving, cost-efficient on-device applications ([codenomad.net](https://www.codenomad.net/blog/what-new-in-generative-ai/?utm_source=openai)).
- Agentic AI is advancing rapidly: AI agents now autonomously browse, interact with tools, and orchestrate multi-step workflows via frameworks such as AutoGPT, LangChain, and OpenAgents ([codenomad.net](https://www.codenomad.net/blog/what-new-in-generative-ai/?utm_source=openai)).
- Orchestration platforms are coordinating multiple generative models into efficient pipelines, enabling model routing, context sharing, and error correction across agentic services ([futureagi.com](https://futureagi.com/blogs/generative-ai-trends-2025?utm_source=openai)).
- Open-source initiatives are democratizing access, with frameworks like Hugging Face Transformers and Stable Diffusion WebUI making GenAI tools more accessible to individuals and startups ([futureagi.com](https://futureagi.com/blogs/generative-ai-trends-2025?utm_source=openai)).
- Increasing focus on human-in-the-loop systems ensures ethical and aligned AI outputs, with Copilot being a prominent example ([cloud-awards.com](https://www.cloud-awards.com/how-gen-ai-will-revolutionize-2025?utm_source=openai)).

---

## Summary

In November 2025, generative AI is reaching new maturity across platforms and industries:

- **Model Innovations**: GPT‑5, Gemini’s Nano Banana, Veo 3 updates, and Meta’s ad-targeting GEM model lead the charge.
- **Industry Integration**: The TIME AI Agent transforms journalism engagement; India’s market growth signals broad commercial adoption.
- **Hardware Momentum**: Qualcomm’s AI200/250 accelerators herald new infrastructure for scalable model deployment.
- **Emerging Frontiers**: Interactive 3D environments (Genie 3), agentic workflows, orchestrated AI pipelines, and ethical safeguards are shaping future capabilities.

These developments reflect a shift toward integrated, autonomous, and responsible generative AI systems impacting creativity, business, and society.

Let me know if you'd like a deeper dive into any specific area or model!

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s an in-depth overview of the **latest developments in generative AI** as of **November 10, 202

### Continue Query with Web Search

In [17]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [18]:
display(Markdown(continue_search_response.output_text))

Here’s a richly detailed roundup of the **latest generative AI developments** as of **November 10, 2025**, based on current news coverage. Each section includes diverse sources and concrete dates to ensure clarity and context.

---

##  Key Recent News in Generative AI

### 1. Microsoft’s MAI‑Image‑1 Launch
- **Event**: On November 5, 2025, Microsoft introduced **MAI‑Image‑1**, a proprietary text-to-image generation model delivered via Bing Image Creator and Copilot Audio. It offers rapid creation of highly detailed, photorealistic visuals (e.g., nature scenes, food). While not yet deployed in the EU, a broader rollout is anticipated soon.([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai))
- **Significance**: This launch marks a strategic move by Microsoft to reduce reliance on OpenAI while retaining collaborative ties, as their licensing agreement for models like DALL·E 3 and GPT‑4o remains active through at least 2032([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai)).

### 2. Adobe’s AI-Powered Creative Suite at Adobe MAX 2025
- **Event**: During Adobe MAX 2025 (late October/early November), Adobe released a major update integrating AI across its product lineup—Express, Firefly, Creative Cloud. Highlights include:
  - **Firefly Image Model 5**, enhancing quality and versatility in image generation.
  - Full video creation capabilities in Firefly.
  - Strengthened AI tools for Photoshop, Premiere Pro, and the new Adobe GenStudio.
  - Collaboration integrating Google’s **Nano Banana** (“Gemini 2.5 Flash Image”) technology.([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/adobe-launches-ai-audio-video-and-photo-tools-across-express-firefly-and-creative-cloud-brings-firefly-image-model-5-googles-nano-banana-and-more/articleshow/124876149.cms?utm_source=openai))
- **Impact**: Adobe is positioning itself more competitively against AI-first startups by embedding powerful generative tools for both creative professionals and casual users([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/tech-news/adobe-launches-ai-audio-video-and-photo-tools-across-express-firefly-and-creative-cloud-brings-firefly-image-model-5-googles-nano-banana-and-more/articleshow/124876149.cms?utm_source=openai)).

### 3. Ethical Challenges of AI-Generated Imagery
- **Event**: On November 10, 2025, ACT Alliance and Fairpicture— a Swiss startup—highlighted the ethical risks associated with fully AI-generated images. In a webinar titled *“A crisis of compassion? Ethically navigating the current fundraising landscape,”* they argued that such images can degrade the dignity of communities featured in visual communications.([actalliance.org](https://actalliance.org/act-news/fully-ai-generated-images-hurt-the-dignity-of-communities/?utm_source=openai))
- **Implication**: The discussion emphasizes the growing need for ethical considerations and human-centered design in generative media, especially in sensitive sectors like fundraising and journalism([actalliance.org](https://actalliance.org/act-news/fully-ai-generated-images-hurt-the-dignity-of-communities/?utm_source=openai)).

### 4. Google’s AI Training Support for Veterans
- **Event**: On November 10, 2025, Google unveiled a new **Generative AI Leader certification program** tailored for military veterans. Scheduled for November 13–14, this program builds on training initiatives that have already reached over 23,000 service members and contractors. It’s part of a broader effort to bridge veterans into AI careers via Google Cloud, Google Skills, and Launchpad for Veterans.([startuphub.ai](https://www.startuphub.ai/ai-news/ai-research/2025/google-accelerates-veteran-ai-training-initiatives/?utm_source=openai))
- **Significance**: This represents a notable step towards responsible AI workforce development, focusing on inclusivity and equipping diverse talent with in-demand skills([startuphub.ai](https://www.startuphub.ai/ai-news/ai-research/2025/google-accelerates-veteran-ai-training-initiatives/?utm_source=openai)).

### 5. Legal and Regulatory Developments in AI
- **Event**: The week of November 3–7, 2025, saw several notable legal updates:
  - The **UK High Court** dismissed Getty’s secondary copyright lawsuit against **Stability AI**.
  - **An Indian court** lifted a data-sharing ban on WhatsApp.
  - **Sweden** launched an investigation into a data leak attributed to a software supplier.([lexology.com](https://www.lexology.com/pro/content/data-protection-key-compliance-updates-3-7-nov?utm_source=openai))
- **Relevance**: These developments underscore evolving regulatory tensions affecting generative AI—especially around content rights, data governance, and privacy compliance([lexology.com](https://www.lexology.com/pro/content/data-protection-key-compliance-updates-3-7-nov?utm_source=openai)).

### 6. Broader Trends and Infrastructure Growth
- **Event**: November 10, 2025: Market attention is sharply focused on the **semiconductor surge** fueled by AI demand. The industry is experiencing an unprecedented flood of investment, with projections pointing toward a trillion-dollar valuation by the end of the decade.([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/tokenring-2025-11-10-the-ai-gold-rush-semiconductor-investments-soar-amidst-global-tech-transformation?utm_source=openai))
- **Significance**: This investment boom is foundational for generative AI’s scalability, as hardware—especially specialized chips—is critical for training and deploying advanced models([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/tokenring-2025-11-10-the-ai-gold-rush-semiconductor-investments-soar-amidst-global-tech-transformation?utm_source=openai)).

---

##  Summary Table

| Area                    | Recent Development                                                                              | Date                |
|-------------------------|--------------------------------------------------------------------------------------------------|---------------------|
| Proprietary AI Models   | Microsoft launches MAI‑Image‑1                                                                  | Nov 5, 2025         |
| Creative Tools          | Adobe MAX: Firefly 5, video tools, Nano Banana integration                                      | Late Oct–Early Nov 2025 |
| Ethical Use             | Fairpicture & ACT Alliance raising concerns on AI-generated images’ dignity implications         | Nov 10, 2025        |
| Workforce Development   | Google unveils AI certification program for veterans                                             | Nov 10, 2025        |
| Legal/Regulatory        | Getty vs Stability AI, WhatsApp ban lifted, Sweden data leak investigation                      | Week of Nov 3–7, 2025 |
| Hardware Infrastructure | Semiconductor investment boom driven by AI demand                                               | Nov 10, 2025        |

---

If you're interested in diving deeper into any of these stories—whether it's model capabilities, ethical dimensions, market impacts, or policy implications—feel free to ask.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [19]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, model releases, market trends, and emerging applications.

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model that integrates reasoning and non-reasoning capabilities under a unified interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and represents a significant leap in performance across benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **OpenAI GPT‑4.1 and o4‑mini**  
  - **GPT‑4.1** launched on **April 14, 2025**, offering improved coding capabilities and available in mini and nano variants for broader accessibility. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai))  
  - **o4‑mini**, released on **April 16, 2025**, is a lightweight multimodal model capable of processing text and images, including whiteboard sketches, and is available to all ChatGPT users. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai))

- **Google DeepMind’s Gemini Series & “Nano Banana”**  
  - The **Gemini 2.5** family, including Flash and Pro variants, introduced advanced reasoning, coding, and “Deep Think” capabilities, with general availability announced on **June 17, 2025**. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai))  
  - **Nano Banana** (Gemini 2.5 Flash Image), launched on **August 26, 2025**, is a viral image generation and editing model known for photorealistic “3D figurine” outputs, subject consistency, multi-image fusion, and SynthID watermarking. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Other Notable Models**  
  - **GPT‑4.5 (Orion)** debuted in **February 2025**, offering enhanced multimodal capabilities, faster inference, and a massive 256K token context window. ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai))  
  - **xAI’s Grok 3 and Grok 3 Mini**, **Anthropic’s Claude 3.7 Sonnet**, and **Baidu’s Ernie 4.5 and Ernie X1** also launched earlier in 2025, each bringing unique strengths in reasoning, self-reflection, and open-source accessibility. ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai))

---

##  Emerging Trends and Market Dynamics

- **Agentic AI and Autonomous Agents**  
  Enterprises are increasingly adopting agentic AI—systems capable of planning and executing multi-step workflows autonomously. According to McKinsey’s 2025 Global AI Survey, 23% of organizations are scaling agentic AI, while 39% are experimenting with it. High performers are redesigning workflows and investing heavily in AI, with over one-third allocating more than 20% of their digital budgets to AI. ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai))

- **Market Growth and Infrastructure Investment**  
  The generative AI market is experiencing triple-digit growth across hardware, foundation models, and development platforms. Cloud providers are investing heavily in data centers, with AI-related spending expected to exceed **US$400 billion in 2025**. ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai))

- **Responsible AI Practices**  
  UC Berkeley’s Responsible Use of Generative AI playbook outlines ten actionable strategies—five for business leaders and five for product managers—to ensure ethical and responsible deployment of GenAI in products and workflows. ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai))

---

##  Recent Product Launches and Applications

- **TIME AI Agent**  
  Launched **today**, the TIME AI Agent is a generative AI platform developed with Scale AI. It enables interactive news consumption through summaries, audio reports, translations, and conversational exploration, while maintaining editorial integrity via robust moderation and data governance. ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **Microsoft’s MAI‑Image‑1**  
  Released **five days ago**, MAI‑Image‑1 is Microsoft’s proprietary text-to-image generator, integrated into Bing Image Creator and Copilot Audio Expressions. It offers fast, photorealistic image generation and greater creative control, while Microsoft continues to maintain integration rights with OpenAI technologies through 2032. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai))

- **Weekly AI Highlights (Nov 1–7, 2025)**  
  - **Sora for Android** achieved nearly **470,000 downloads** on its first day (Nov 6), outperforming its iOS launch. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - **Google’s DS STAR**, a multi-agent framework, can translate ambiguous business problems into executable Python code without human intervention. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - **Meta’s Vibes** platform expanded to Europe, offering AI-generated short videos akin to TikTok or Reels. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - **Google Maps** is integrating Gemini as a voice assistant for intuitive navigation. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - **HeyGen** launched a hyper-realistic AI video translator with lifelike localization. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - A **Gemini 3 Pro preview** appeared on Vertex AI, hinting at a model with a 1-million-token context window. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))  
  - **ClickUp 4.0** debuted with AI agents and a redesigned UI, consolidating task management, collaboration, and search. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  Summary: What’s Driving the Generative AI Landscape

1. **Model Evolution**: The field is advancing rapidly with GPT‑5, Gemini 2.5, Nano Banana, and other models pushing the boundaries of multimodal reasoning and creativity.
2. **Enterprise Adoption**: Agentic AI is moving from experimentation to scaling, supported by significant budget allocations and workflow redesigns.
3. **Market Expansion**: Generative AI is becoming core infrastructure, with massive investments and triple-digit growth across sectors.
4. **Ethical Deployment**: Responsible AI frameworks are gaining prominence to guide safe and trustworthy adoption.
5. **Real-World Integration**: AI is increasingly embedded in everyday tools—from news platforms and search to navigation and productivity apps.

---

If you'd like to explore any of these developments in more detail—such as technical capabilities of specific models, enterprise use cases, or ethical frameworks—feel free to ask!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [21]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6912675da020819198fc1449ab0eaebe


In [22]:
with open('Character_ Responsibilities.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-Mi8oK6VuSmQgF3Qi4zkKKQ


In [23]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-Mi8oK6VuSmQgF3Qi4zkKKQ


In [24]:
query = "What is responsibilites?"

In [25]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

chCharacter: Responsibilities
1. “Ethical Responsibility"
 https://fiveable.me/key-terms/ethics/et
 Relevant score: 0.8223943512607178


In [26]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [27]:
display(Markdown(simple_response.output_text))

"Responsibilities" refer to the duties or obligations a person is expected to fulfill in a particular role or situation. These can vary widely based on context, such as professional responsibilities at work, personal responsibilities in family or relationships, and civic responsibilities as a member of a community or society. Fulfilling responsibilities typically involves taking ownership of tasks, being accountable for one's actions, and ensuring that obligations are met effectively and ethically.

## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [28]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [29]:
display(Markdown(web_search_response.output_text))

"Responsibilities" refer to the duties or obligations that a person is expected to fulfill. These can arise from various roles in life, such as personal, professional, or social roles. Responsibilities often involve being accountable for actions and the outcomes they produce. For example, a parent has the responsibility to care for their children, and an employee has the responsibility to complete assigned tasks effectively.

In [33]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

"Responsibilities" refer to the duties or obligations that a person is expected to fulfill. These ca

In [34]:
continue_query = 'find different meanings of responsibilites'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [35]:
display(Markdown(continue_search_response.output_text))

Here are various meanings of "responsibilities":

1. **Accountability**: Being answerable for one's actions and the outcomes of those actions.
   
2. **Obligations**: Duties that a person is expected to perform, arising from roles, agreements, or laws.

3. **Trustworthiness**: The quality of being reliable and dependable in fulfilling duties.

4. **Moral or Ethical Duty**: An obligation to act in a way that is right or just, often guided by personal or societal values.

5. **Legal Liabilities**: Responsibilities imposed by law, where failure to fulfill them can result in legal consequences.

6. **Social Roles**: Expectations tied to one's position in social or professional contexts, such as being a responsible family member or community leader.

7. **Functional Roles**: Tasks or activities associated with a particular job or position that ensure successful operation.

In [36]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [37]:
display(Markdown(combined_search_response.output_text))

Responsibilities refer to the obligations or duties that individuals or groups have towards others or organizations. They emphasize accountability and can impact the welfare of others if not fulfilled. Responsibilities can be ethical, social, or moral, and they often involve making choices that respect others and society as a whole.